## 상품정보 데이터

### json데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## items

In [ ]:
items = pd.read_json('./items.json')
items.head()

In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [ ]:
review = pd.read_json('./reviews.json')
review.info()


In [ ]:
print(list(review))

In [ ]:
# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '선택1', '타입', '리뷰', '사이즈', 'color', 'size', '컬러', '색상-사이즈', '색상']] # '데님', '타입2'

In [ ]:
review.head()

In [ ]:
# NaN -> None
review = review.replace({np.nan: None})
review.info()

### 카디널리티 확인

In [ ]:
for col in ['별점', '키', '몸무게', '평소사이즈']:
    print(col)
    print(review[col].unique())
    print('\n')

* 핫핑 사이즈 기준

<img src='./imgs/사이즈통일.png'>

In [ ]:
# 평소 사이즈 수정
key1 = [str(i) for i in range(44, 100, 11)] +['100']
key2 = ['S', 'M', 'L', 'XL', 'XXL', 'XXXL']

def user_size(row):
    for i in range(1,6):
        if row == key1[i]:
            return str(i)
        elif row == key2[i]:
            return str(i)
        
review['평소사이즈'] = review['평소사이즈'].apply(user_size)
review['평소사이즈'].unique()

In [ ]:
# 리뷰 데이터 줄바꿈 -> 공백으로 대체
def review_comm(row):
    return re.sub('\n', ' ', row)

review['리뷰'] = review['리뷰'].apply(review_comm)
review.head()

### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [ ]:
# review.loc[review['사이즈']=='핑크']

In [ ]:
# # 값 서로 바꾸기
# idx = review.loc[review['사이즈']=='핑크'].index
# review.loc[idx, '사이즈'], review.loc[idx, '컬러'] = review.loc[idx, '컬러'], review.loc[idx, '사이즈']
# review.iloc[idx, :]

In [25]:
# 카디널리티 확인
cols = ['선택1', '타입', '사이즈', 'color', 'size', '컬러', '색상-사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

선택1
[None '브라운(brown)-F' '화이트(white)-F' '블랙(black)-F' '멜란지그레이(m.grey)-F'
 '소라(L.teal)-F' '카키(khaki)-F' '라이트핑크(L.pink)-F' '옐로우(yellow)-F'
 '오렌지(orange)-F' '연베이지(L.beige)-F' '그린(green)-F' '올리브(olive)-F']


타입
[None '화이트(white)-F' '블루(blue)-F' '차콜(charcoal)-F' '블랙(black)-2'
 '아이보리(ivory)-2' '블랙(black)-3' '아이보리(ivory)-3' '민트(mint)-3'
 '아이보리(ivory)-1' '민트(mint)-2' '블랙(black)-1' '민트(mint)-1' '라이트핑크(L.pink)-1'
 '라이트핑크(L.pink)-3' '라이트핑크(L.pink)-2' '그린(green)-1' '그린(green)-3'
 '그린(green)-2' '라이트민트(L.mint)-3' '소라(L.teal)-2' '소라(L.teal)-3'
 '라이트민트(L.mint)-2' '핑크(pink)-2' '핑크(pink)-1' '핑크(pink)-3'
 '라이트민트(L.mint)-1' '소라(L.teal)-1' '라이트핑크(L.pink)-F' '아이보리(ivory)-F'
 '모카(moca)-F' '블랙(black)-F' '퍼플(purple)-F' '오렌지(orange)-F' '옐로우(yellow)-F'
 '민트(mint)-F' '딥그린(D.green)-2' '딥그린(D.green)-3' '연베이지(L.beige)-3'
 '브라운(brown)-2' '브라운(brown)-3' '브라운(brown)-1' '딥그린(D.green)-1'
 '연베이지(L.beige)-2' '연베이지(L.beige)-1' '코코아(cocoa)-3' '차콜(charcoal)-1'
 '백염(white salt)-3' '차콜(charcoal)-2' '차콜(charcoal)-3' '백염(white sa

In [26]:
drop = review.loc[review.컬러 == '화그블'].index
review.drop(drop, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)

In [27]:
def eng_kor(row):
    if row:
        return '블랙'

review['color'] = review['color'].apply(eng_kor)
review.color.unique()

array([None, '블랙'], dtype=object)

In [28]:
def float_str(row):
    if row == 1.:
        return '1'
review['size'] = review['size'].apply(float_str)
review['size'].unique()

array([None, '1'], dtype=object)

#### 사이즈 있는경우 '색상-사이즈', 없는 경우 '색상' 형태로 값 통일

In [29]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free' or row=='프리':
        return 'F'
    elif row=='M':
        return '2'
    
review['사이즈'] = review['사이즈'].apply(size)

# review.loc[review['사이즈']=='블랙 1', '사이즈'] = '블랙-1'

# 컬러 컬럼
def color(row):
    if row: # Nan => None
        if '_' in row:
            return row.split('_')[-1]
        elif '/' in row:
            return row.split('/')[-1]
        elif '미니-' in row:
            row = row.replace('미니-', '')
            return row
        elif ' ' in row and '(' not in row:
            return row.split()[-1]
        elif ',' in row:
            return row.split(',')[-1].strip()
        elif row == '블':
            return '블루'
        else:
            return row
    
# review['컬러'] = review['컬러'].apply(color)


# 타입 컬럼
def type_prc(row):
    if not row: # None값 제외
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif ' [예약판매]' in row:
        row = row.replace(' [예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif row[0].isdigit() and ' ' not in row: # return T/F
        return None
    elif row[0].isdigit() and ' ' in row:
        return row.split('-')[-1]
    elif '(' not in row: # 타입과 무관한값은 ()문자가 없음
        return None
    else:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

# review['데님'] = review['데님'].apply(denim_color)
# review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
# review['타입2'] = review['타입2'].apply(denim_color)

In [31]:
# 카디널리티 확인
cols = ['선택1', '타입', '사이즈', 'color', 'size', '컬러', '색상-사이즈'] #, '색상-사이즈', '데님'
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

선택1
[None '브라운(brown)-F' '화이트(white)-F' '블랙(black)-F' '멜란지그레이(m.grey)-F'
 '소라(L.teal)-F' '카키(khaki)-F' '라이트핑크(L.pink)-F' '옐로우(yellow)-F'
 '오렌지(orange)-F' '연베이지(L.beige)-F' '그린(green)-F' '올리브(olive)-F']


타입
[None '화이트(white)-F' '블루(blue)-F' '차콜(charcoal)-F' '블랙(black)-2'
 '아이보리(ivory)-2' '블랙(black)-3' '아이보리(ivory)-3' '민트(mint)-3'
 '아이보리(ivory)-1' '민트(mint)-2' '블랙(black)-1' '민트(mint)-1' '라이트핑크(L.pink)-1'
 '라이트핑크(L.pink)-3' '라이트핑크(L.pink)-2' '그린(green)-1' '그린(green)-3'
 '그린(green)-2' '라이트민트(L.mint)-3' '소라(L.teal)-2' '소라(L.teal)-3'
 '라이트민트(L.mint)-2' '핑크(pink)-2' '핑크(pink)-1' '핑크(pink)-3'
 '라이트민트(L.mint)-1' '소라(L.teal)-1' '라이트핑크(L.pink)-F' '아이보리(ivory)-F'
 '모카(moca)-F' '블랙(black)-F' '퍼플(purple)-F' '오렌지(orange)-F' '옐로우(yellow)-F'
 '민트(mint)-F' '딥그린(D.green)-2' '딥그린(D.green)-3' '연베이지(L.beige)-3'
 '브라운(brown)-2' '브라운(brown)-3' '브라운(brown)-1' '딥그린(D.green)-1'
 '연베이지(L.beige)-2' '연베이지(L.beige)-1' '코코아(cocoa)-3' '차콜(charcoal)-1'
 '백염(white salt)-3' '차콜(charcoal)-2' '차콜(charcoal)-3' '백염(white sa

In [ ]:
review.info()

In [ ]:
# size_idx = review.loc[review['size'].notnull()==True].index
# for i in size_idx:
#     review.loc[i, '사이즈'] = '1'

In [34]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.loc[i, col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)[0]
        # print(row)
    return df

# # 사이즈 컬럼 별도 수행
# s_idx = review.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
# for s in s_idx:
#     review.loc[s, ['색상', '사이즈']] = review.iloc[s]['사이즈'].split('-')
        
cols = ['선택1', '타입', 'color', '컬러', '색상-사이즈', '색상'] # , '색상-사이즈', '데님'
review = col_merge(review, cols)
size_idx = review.loc[review['size'].notnull()==True].index
for i in size_idx:
    review.loc[i, '사이즈'] = review.loc[i, 'size']
review.sample(5)

,품번,아이디,별점,키,몸무게,평소사이즈,선택1,타입,리뷰,사이즈,color,size,컬러,색상-사이즈,색상
1664,635,1767992,5,166~170,65~,3,옐로우(yellow)-F,None,노랑은 예쁜데 초록은 촌스런 그린 ...... 그냥 싸게사서 입우려고 합니다 무난한...,F,None,None,None,None,옐로우
10216,645,rmsg,5,151~155,65~,3,None,블랙(black)-F,진짜 제일 마음에 든 티셔츠 다른 색도 구매하고 싶어요 오자마자 신나서 입고 사진 ...,F,None,None,None,None,블랙
5598,641,2661412,5,171~175,65~,4,None,블랙(black)-3,낭낭해서 좋아요 큰편이어니라 붙는스타일이에요 날씬해 보여요 가을에 입어도 좋을거같아요,3,None,None,None,None,블랙
16372,652,tha,5,156~160,65~,4,None,None,나시 정말 필요했는데 잘 맞네요~!!!! 원하던 색상은 블블화인데 그건 없어서 아쉬...,1,None,None,None,None,None
6,630,jeong,5,156~160,61 ~ 65,3,None,차콜(charcoal)-F,지금 입기에 완전 딱! 얇아요! 그래서 손이 자주 갔어요! 사이즈도 완전 넉넉! 엉...,F,None,None,None,None,차콜


In [ ]:
review.info()

* 카디널리티 확인

In [36]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
[None 'F' '1' '3' '2' '4']


색상
[None '먹색' '차콜' '화이트' '블루' '민트' '블랙' '아이보리' '라이트핑크' '그린' '브라운' '멜란지그레이'
 '소라' '카키' '옐로우' '라이트민트' '핑크' '오렌지' '연베이지' '올리브' '모카' '퍼플' '딥그린' '코코아'
 '백염' '크림' '네이비' '백멜란지' '그레이' '연그레이' '스카이' '라이트옐로우' '연카키' '라이트오렌지' '레드'
 '머스타드' '베이지' '화블' '화화' '블블' '데님' '딥뮤트블루' '오트베이지']




In [35]:
def sel_size(row):
    size = ['S', 'M', 'L', 'XL', 'XL']
    size_dict = {size[i-1]: str(i) for i in range(1,6)}
    if row:
        if row.isdigit() or row=='F':
            return row
        elif row == ' F':
            return row.strip()
        else:
            return size_dict[row]

review['사이즈'] = review['사이즈'].apply(sel_size)
review['사이즈'].unique()

array([None, 'F', '1', '3', '2', '4'], dtype=object)

In [37]:
# 현재까지 처리된 df 사본
review_prc = review.copy()

#### 사이즈 컬럼 추가 전처리

In [ ]:
# def last_size(row):
#     if row:
#         if row == 'L':
#             return '3'
#         elif row == 'B)':
#             return None
#         elif row == ' F':
#             return 'F'
#         else:
#             return row
# review_prc['사이즈'] = review_prc['사이즈'].apply(last_size)

# review_prc['사이즈'].unique()

            

* 나머지 컬럼은 삭제

In [ ]:
review_prc.info()

In [38]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review_prc.drop(columns=['타입','size', '컬러', '색상-사이즈'], inplace=True)

### '별점' 컬럼 핫핑 직원 데이터는 제외

In [40]:
print(review['아이디'].unique())

['핫핑스탭' '스탭리뷰' '핫핑스텝' ... 'chane' '225368877' 'jis3']


In [42]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review_prc['아이디'].unique():
    r = p.search(str(name))
    if r:
        staffs.add(name)
print(staffs)

{'핫핑-수진', '핫핑-가원', '핫핑스탭', '핫핑-소현', '핫핑-나영', '핫핑-채은', '핫핑 - 수연', '핫핑-다슬', '핫핑-효진', '핫핑-현진', '스탭리뷰', '핫핑-진', '핫핑-정유', '핫핑-혜원', '핫핑-하린', '핫핑-유리', '핫핑_yj', '핫핑-모모', '핫핑_MJ', '핫핑-연재', '핫핑스텝', '핫핑_B.F', '핫핑_수진'}


In [43]:
for s in staffs:
    review_prc.loc[review_prc['아이디'] == s, '별점'] = -1 # 분석에 사용하지 않음

In [ ]:
review_prc.info()

In [44]:
review_prc['별점'].value_counts()

별점
 5    16538
 4      882
 3      480
 1      163
 2      156
-1       78
Name: count, dtype: int64

In [45]:
review_prc.to_json('../Load/prc_data/reviews_prc_4.json', orient='records', indent=2, force_ascii=False)